In [36]:
import pandas as pd

nutrientes_df = pd.read_csv("../data/nutrientes_alimentos.csv")
porciones_df = pd.read_csv("../data/food_portion.csv", usecols=["portion_description", "gram_weight"])
nutrientes_df.head()


,Alimento,Grupo,Porción,Gramos,"Calcium, Ca",Retinol,Lycopene,MUFA 20:1,PUFA 20:5 n-3 (EPA),SFA 14:0,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
0,"Milk, NFS","Milk, reduced fat",1 cup,NaN,125.0,57.0,0.0,0.002,0.000,0.204,...,1.9,58.0,0.0,1.0,39.0,0.1,0.0,0.208,0.465,0.046
1,"Milk, whole","Milk, whole",1 cup,NaN,123.0,31.0,0.0,0.004,0.001,0.303,...,1.9,32.0,0.0,0.0,38.0,0.0,0.0,0.309,0.694,0.067
2,"Milk, reduced fat (2%)","Milk, reduced fat",1 cup,NaN,126.0,83.0,0.0,0.002,0.000,0.181,...,1.8,83.0,0.0,2.0,39.0,0.2,0.0,0.184,0.410,0.041
3,"Milk, low fat (1%)","Milk, lowfat",1 cup,NaN,126.0,58.0,0.0,0.001,0.000,0.093,...,2.1,58.0,0.0,2.0,39.0,0.0,0.0,0.096,0.214,0.022
4,"Milk, fat free (skim)","Milk, nonfat",1 cup,NaN,132.0,64.0,0.0,0.000,0.000,0.008,...,2.0,64.0,0.0,2.0,41.0,0.0,0.0,0.009,0.018,0.003


In [37]:
porciones_df.head(15)

,portion_description,gram_weight
0,1 cup,246.0
1,Quantity not specified,0.0
2,1 fl oz,30.8
3,Guideline amount per cup of hot cereal,61.0
4,1 individual school container,244.0
5,Quantity not specified,244.0
6,1 fl oz,30.5
7,1 cup,244.0
8,Guideline amount per fl oz of beverage,2.5
9,Guideline amount per fl oz of beverage,2.5


In [ ]:
#normalizar texto
porciones_df["portion_description"] = porciones_df["portion_description"].astype(str).str.strip().str.lower()
nutrientes_df["Porción"] = nutrientes_df["Porción"].astype(str).str.strip().str.lower()

#eliminar registros con gram_weight = 0 o NaN vacio
porciones_df = porciones_df[porciones_df["gram_weight"].notna() & (porciones_df["gram_weight"] > 0)]

#eliminar duplicados para tener un mapeo único por porción
porciones_df = porciones_df.drop_duplicates(subset=["portion_description"])
porciones_df

,portion_description,gram_weight
0,1 cup,246.0
2,1 fl oz,30.8
3,guideline amount per cup of hot cereal,61.0
4,1 individual school container,244.0
5,quantity not specified,244.0
...,...,...
21930,1 can or bottle (32 fl oz),992.0
21932,1 can or bottle (8 fl oz),248.0
21933,1 can or bottle (24 fl oz),744.0
21998,1 bottle (32 fl oz),992.0


In [ ]:
#  diccionario: porción → gramos
mapa_porciones = dict(zip(porciones_df["portion_description"], porciones_df["gram_weight"]))

#reemplazar Gramos NaN o igual a 0
nutrientes_df["Gramos"] = nutrientes_df.apply(
    lambda row: mapa_porciones.get(row["Porción"], row["Gramos"]) 
    if (pd.isna(row["Gramos"]) or row["Gramos"] == 0) else row["Gramos"],
    axis=1
)


In [40]:
nutrientes_df

,Alimento,Grupo,Porción,Gramos,"Calcium, Ca",Retinol,Lycopene,MUFA 20:1,PUFA 20:5 n-3 (EPA),SFA 14:0,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
0,"Milk, NFS","Milk, reduced fat",1 cup,246.0,125.0,57.0,0.0,0.002,0.000,0.204,...,1.9,58.0,0.0,1.0,39.0,0.1,0.0,0.208,0.465,0.046
1,"Milk, whole","Milk, whole",1 cup,246.0,123.0,31.0,0.0,0.004,0.001,0.303,...,1.9,32.0,0.0,0.0,38.0,0.0,0.0,0.309,0.694,0.067
2,"Milk, reduced fat (2%)","Milk, reduced fat",1 cup,246.0,126.0,83.0,0.0,0.002,0.000,0.181,...,1.8,83.0,0.0,2.0,39.0,0.2,0.0,0.184,0.410,0.041
3,"Milk, low fat (1%)","Milk, lowfat",1 cup,246.0,126.0,58.0,0.0,0.001,0.000,0.093,...,2.1,58.0,0.0,2.0,39.0,0.0,0.0,0.096,0.214,0.022
4,"Milk, fat free (skim)","Milk, nonfat",1 cup,246.0,132.0,64.0,0.0,0.000,0.000,0.008,...,2.0,64.0,0.0,2.0,41.0,0.0,0.0,0.009,0.018,0.003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5426,Vegetables as ingredient in curry,Not included in a food category,no especificado,NaN,21.0,0.0,458.0,0.000,0.000,0.002,...,0.4,98.0,0.0,27.0,20.0,17.1,0.0,0.006,0.016,0.000
5427,Vegetables as ingredient in soups,Not included in a food category,no especificado,NaN,17.0,0.0,764.0,0.000,0.000,0.001,...,0.2,38.0,0.0,26.0,15.0,21.0,0.0,0.006,0.034,0.000
5428,Vegetables as ingredient in stews,Not included in a food category,no especificado,NaN,21.0,0.0,1088.0,0.000,0.000,0.001,...,0.2,259.0,0.0,25.0,34.0,14.0,0.0,0.004,0.019,0.000
5429,Sauce as ingredient in hamburgers,Not included in a food category,no especificado,NaN,18.0,4.0,7237.0,0.106,0.000,0.017,...,4.5,21.0,0.0,8.0,845.0,2.5,0.0,0.937,5.038,0.000


In [41]:
nutrientes_df[nutrientes_df["Gramos"].isna() | (nutrientes_df["Gramos"] == 0)]
nutrientes_df.tail()


,Alimento,Grupo,Porción,Gramos,"Calcium, Ca",Retinol,Lycopene,MUFA 20:1,PUFA 20:5 n-3 (EPA),SFA 14:0,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
5426,Vegetables as ingredient in curry,Not included in a food category,no especificado,NaN,21.0,0.0,458.0,0.000,0.0,0.002,...,0.4,98.0,0.0,27.0,20.0,17.1,0.0,0.006,0.016,0.0
5427,Vegetables as ingredient in soups,Not included in a food category,no especificado,NaN,17.0,0.0,764.0,0.000,0.0,0.001,...,0.2,38.0,0.0,26.0,15.0,21.0,0.0,0.006,0.034,0.0
5428,Vegetables as ingredient in stews,Not included in a food category,no especificado,NaN,21.0,0.0,1088.0,0.000,0.0,0.001,...,0.2,259.0,0.0,25.0,34.0,14.0,0.0,0.004,0.019,0.0
5429,Sauce as ingredient in hamburgers,Not included in a food category,no especificado,NaN,18.0,4.0,7237.0,0.106,0.0,0.017,...,4.5,21.0,0.0,8.0,845.0,2.5,0.0,0.937,5.038,0.0
5430,Industrial oil as ingredient in food,Not included in a food category,no especificado,NaN,0.0,0.0,0.0,0.322,0.0,0.048,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.263,30.664,0.0


In [42]:
nutrientes_df = nutrientes_df[~nutrientes_df["Gramos"].isna() & (nutrientes_df["Gramos"] > 0)]


In [43]:
nutrientes_df.tail()

,Alimento,Grupo,Porción,Gramos,"Calcium, Ca",Retinol,Lycopene,MUFA 20:1,PUFA 20:5 n-3 (EPA),SFA 14:0,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
5389,"Sports drink, low calorie (Powerade Zero)",Diet sport and energy drinks,1 cup,246.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,42.0,0.0,0.0,0.0,0.0,0.0
5390,"Sports drink, low calorie",Diet sport and energy drinks,1 cup,246.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,45.0,0.0,0.0,0.0,0.0,0.0
5391,"Fluid replacement, electrolyte solution",Sport and energy drinks,quantity not specified,244.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,101.0,0.0,0.0,0.0,0.0,0.0
5392,"Fluid replacement, 5% glucose in water",Sport and energy drinks,1 cup,246.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,9.0,0.0,0.0,0.0,0.0,0.0
5393,"Fruit juice, acai blend",Fruit drinks,quantity not specified,244.0,19.0,0.0,899.0,0.0,0.0,0.0,...,0.0,485.0,11.11,4.0,13.0,42.1,0.0,0.0,0.0,0.0


agrupar mejor los grupos de alimentos

In [ ]:
grupo_alimentos_large = nutrientes_df[["Grupo"]].drop_duplicates()
grupo_alimentos_large.to_csv("grupo_alimentos_large.csv", index=False)

In [45]:
#extraer la columna 'Alimento' de alimento_gramos.csv
#grupo_alimentos_small = pd.read_csv('../data_final/consumo_dietas.csv', usecols=['Alimento']).drop_duplicates()
#exportar a CSV
#grupo_alimentos_small.to_csv('grupo_alimentos_small.csv', index=False)

bueno de aca pasamos hacer el agrupamiento en el ipynb de agrupamiento y salio el csv llamado AGRUPAMIENTO_FINAL.csv 

In [ ]:
#editar los grupos de alimentos
#cargar el CSV de agrupamiento final
agrup_df = pd.read_csv("agrupamiento_total.csv")  #Grupo, categoria_es

#hacer merge para añadir la categoría_es a nutrientes_df
merged = nutrientes_df.merge(
    agrup_df[['Grupo', 'categoria_es']],
    how='left',
    on='Grupo'
)

In [ ]:

#filtrar solo los registros que sí tengan categoría asignada
filtered = merged[merged['categoria_es'].notna()].copy()

#sobrescribir la columna Grupo con la nueva categoría
filtered['Grupo'] = filtered['categoria_es']

#eliminar la columna auxiliar categoria_es
filtered = filtered.drop(columns=['categoria_es'])

filtered.to_csv("../data/nutrientes_alimentos_agrupados.csv", index=False, encoding="utf-8-sig")
filtered.tail()

,Alimento,Grupo,Porción,Gramos,"Calcium, Ca",Retinol,Lycopene,MUFA 20:1,PUFA 20:5 n-3 (EPA),SFA 14:0,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
5324,"Water, bottled, plain",Agua,1 cup,246.0,10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
5325,"Water, non-carbonated, flavored",Agua,1 cup,246.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0
5326,"Water beverage, fruit flavored",Agua,1 cup,246.0,17.0,32.0,0.0,0.0,0.0,0.0,...,0.0,32.0,1.9,8.0,8.0,12.7,0.0,0.0,0.0,0.0
5327,"Water, enhanced, regular",Agua,1 cup,246.0,17.0,32.0,0.0,0.0,0.0,0.0,...,0.0,32.0,1.9,8.0,0.0,12.7,0.0,0.0,0.0,0.0
5328,"Water, enhanced, diet",Agua,1 cup,246.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.2,0.0,0.0,0.0,0.0


ahora se va agregar el precio por alimentos

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/nutrientes_alimentos_agrupados.csv")

#diccionario de precios por categoría (100 g)
precio100g = {
    "Leche": 0.59, "Yogurt": 0.70, "Batidos": 1.50, "Helado": 1.60,
    "Queso fresco": 2.90, "Huevos": 0.79, "Carne de res": 2.49,
    "Embutidos": 1.80, "Cerdo": 3.20, "Mantequilla": 3.50,
    "Cordero": 3.50, "Pollo": 0.88, "Arroz": 0.52, "Pasta": 0.50,
    "Pan": 0.67, "Cereales": 1.20, "Legumbres": 0.60, "Papas": 0.30,
    "Tomate": 0.40, "Otros vegetales": 0.40, "Lechuga": 2.00,
    "Aceitunas": 1.50, "Manzana": 0.50, "Plátano": 0.35,
    "Cítricos": 0.30, "Jugo": 0.80, "Azúcar": 0.45, "Café": 10.00,
    "Gaseosa": 0.50, "Agua": 0.35, "Cerveza": 0.90, "Vino": 2.50,
    "Durazno": 1.00, "Frutos secos": 4.00, "Galletas": 1.30
}

df["precio100g"] = df["Grupo"].map(precio100g)

#factor de variación aleatoria entre 0.95 y 1.05
np.random.seed(42)
df["factor_var"] = np.random.uniform(0.95, 1.05, size=len(df))

#calcular precio por porción
df["Precio"] = (df["precio100g"] * df["factor_var"] * df["Gramos"] / 100).round(2)

#reordenar columnas para poner precio100g y Precio justo después de Gramos
cols = df.columns.tolist()
i = cols.index("Gramos") + 1
# eliminamos de la lista original para reinsertar en la posición deseada
for col in ["precio100g", "Precio"]:
    cols.remove(col)
new_cols = cols[:i] + ["precio100g", "Precio"] + cols[i:]
df = df[new_cols]

#eliminar columna auxiliar
df = df.drop(columns="factor_var")

df.to_csv(
    "../data_final/nutrientes_alimentos_agrupados_precios.csv",
    index=False,
    encoding="utf-8-sig"
)

In [49]:
df.head()


,Alimento,Grupo,Porción,Gramos,precio100g,Precio,"Calcium, Ca",Retinol,Lycopene,MUFA 20:1,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
0,"Milk, NFS",Leche,1 cup,246.0,0.59,1.43,125.0,57.0,0.0,0.002,...,1.9,58.0,0.0,1.0,39.0,0.1,0.0,0.208,0.465,0.046
1,"Milk, whole",Leche,1 cup,246.0,0.59,1.52,123.0,31.0,0.0,0.004,...,1.9,32.0,0.0,0.0,38.0,0.0,0.0,0.309,0.694,0.067
2,"Milk, reduced fat (2%)",Leche,1 cup,246.0,0.59,1.49,126.0,83.0,0.0,0.002,...,1.8,83.0,0.0,2.0,39.0,0.2,0.0,0.184,0.410,0.041
3,"Milk, low fat (1%)",Leche,1 cup,246.0,0.59,1.47,126.0,58.0,0.0,0.001,...,2.1,58.0,0.0,2.0,39.0,0.0,0.0,0.096,0.214,0.022
4,"Milk, fat free (skim)",Leche,1 cup,246.0,0.59,1.40,132.0,64.0,0.0,0.000,...,2.0,64.0,0.0,2.0,41.0,0.0,0.0,0.009,0.018,0.003


In [51]:
df["kcal_por_g"] = df["Energy"] / df["Gramos"]

cols = list(df.columns)
precio_idx = cols.index("Precio") + 1
cols.remove("kcal_por_g")
cols.insert(precio_idx, "kcal_por_g")
df = df[cols]

df.to_csv("../data_final/nutrientes_alimentos_agrupados_precios.csv", index=False, encoding="utf-8-sig")

df.head()

,Alimento,Grupo,Porción,Gramos,precio100g,Precio,kcal_por_g,"Calcium, Ca",Retinol,Lycopene,...,"Selenium, Se","Vitamin A, RAE","Vitamin E, added","Folate, total","Sodium, Na","Vitamin C, total ascorbic acid",Theobromine,SFA 18:0,MUFA 18:1,SFA 4:0
0,"Milk, NFS",Leche,1 cup,246.0,0.59,1.43,0.211382,125.0,57.0,0.0,...,1.9,58.0,0.0,1.0,39.0,0.1,0.0,0.208,0.465,0.046
1,"Milk, whole",Leche,1 cup,246.0,0.59,1.52,0.247967,123.0,31.0,0.0,...,1.9,32.0,0.0,0.0,38.0,0.0,0.0,0.309,0.694,0.067
2,"Milk, reduced fat (2%)",Leche,1 cup,246.0,0.59,1.49,0.203252,126.0,83.0,0.0,...,1.8,83.0,0.0,2.0,39.0,0.2,0.0,0.184,0.410,0.041
3,"Milk, low fat (1%)",Leche,1 cup,246.0,0.59,1.47,0.174797,126.0,58.0,0.0,...,2.1,58.0,0.0,2.0,39.0,0.0,0.0,0.096,0.214,0.022
4,"Milk, fat free (skim)",Leche,1 cup,246.0,0.59,1.40,0.138211,132.0,64.0,0.0,...,2.0,64.0,0.0,2.0,41.0,0.0,0.0,0.009,0.018,0.003
